<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/PyTorch/Basic/ProtBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtBert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [ ]:
!pip install -q transformers

In [14]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline
from transformers import BertForMaskedLM,GPT2LMHeadModel
import re
import numpy as np
import os
import requests
from tqdm.auto import tqdm

<b>2. Load the vocabulary and ProtBert Model<b>

In [2]:
# orginal Prot_BERT
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = AutoModel.from_pretrained("Rostlab/prot_bert")

In [3]:
# our_AMP_BERT
tokenizer = AutoTokenizer.from_pretrained("/workspace/git/transformers-/examples/language-modeling/test-mlm-BERT-base-AMP-Epoch100-maxLen64/", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("/workspace/git/transformers-/examples/language-modeling/test-mlm-BERT-base-AMP-Epoch100-maxLen64/")

In [4]:
tokenizer.all_special_tokens

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [5]:
tokenizer.vocab

{'S': 10,
 'T': 15,
 'Q': 18,
 'X': 25,
 'B': 27,
 'Z': 28,
 'A': 6,
 'N': 17,
 '[UNK]': 1,
 'G': 7,
 'W': 24,
 '[PAD]': 0,
 'P': 16,
 'Y': 20,
 'H': 22,
 'C': 23,
 'O': 29,
 '[MASK]': 4,
 'U': 26,
 'E': 9,
 '[SEP]': 3,
 'M': 21,
 'K': 12,
 'V': 8,
 'D': 14,
 '[CLS]': 2,
 'F': 19,
 'I': 11,
 'R': 13,
 'L': 5}

<b>3. Load the model into the GPU if avilabile<b>

In [6]:
fm = pipeline('fill-mask', model=model, tokenizer=tokenizer,device=0)

In [7]:
with open('/workspace/git/transformers-/examples/language-modeling/uniprot_sprot_60_withSpace.txt') as f:
    orgSamples=f.readlines()

In [8]:
idx =5
test_masked_input = orgSamples[idx]

In [9]:
print(test_masked_input)

A A A I S C V G S K E C L P K C K A Q G C K S G K C M N K K C K C Y C



In [10]:
fm('A A [MASK] I S C V G S K E C L P K C K A Q G C K S G K C M N K K C K C Y C')

[{'sequence': 'A A A I S C V G S K E C L P K C K A Q G C K S G K C M N K K C K C Y C',
  'score': 0.9403852820396423,
  'token': 6,
  'token_str': 'A'},
 {'sequence': 'A A G I S C V G S K E C L P K C K A Q G C K S G K C M N K K C K C Y C',
  'score': 0.045312393456697464,
  'token': 7,
  'token_str': 'G'},
 {'sequence': 'A A T I S C V G S K E C L P K C K A Q G C K S G K C M N K K C K C Y C',
  'score': 0.007608081679791212,
  'token': 15,
  'token_str': 'T'},
 {'sequence': 'A A V I S C V G S K E C L P K C K A Q G C K S G K C M N K K C K C Y C',
  'score': 0.0032630241475999355,
  'token': 8,
  'token_str': 'V'},
 {'sequence': 'A A S I S C V G S K E C L P K C K A Q G C K S G K C M N K K C K C Y C',
  'score': 0.0015351864276453853,
  'token': 10,
  'token_str': 'S'}]

# 4. GPT-2  Demo

model =  ./test-clm-GPT2-test/

change the path to the model.zip path, then can do peptile generation. 

In [44]:

tokenizer = AutoTokenizer.from_pretrained("/workspace/git/transformers-/examples/language-modeling/test-clm-GPT2-test/", do_lower_case=False)
model = GPT2LMHeadModel.from_pretrained("/workspace/git/transformers-/examples/language-modeling/test-clm-GPT2-test/")

In [45]:
tg = pipeline('text-generation', model=model, tokenizer=tokenizer,device=0)

### GroundTruth Sequence:  A A A I S C V G S K E C L P K C K A Q G C K S G K C M N K K C K C Y C

"A A A I S" 是初始序列  , max_length is the maxLength of generation, the generation can be terminal by the ending token, which means the generated result may less than 60. 

In [56]:
tg("A A A I S", max_length=60, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'A A A I S Q T Q R L S L L T E A F G L I D V T P A G S H F E Q D Q E M A A F V R A L H D Y E K R M S VM A A A K G V E Y'}]

In [51]:
tg("A A A I S", max_length=60, num_return_sequences=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'A A A I S C M S D D S I W N N L V A T A L T P T V F F L S G A L I T A M Q F I Q RM A A A K G V W V T I L G A A L G A I'},
 {'generated_text': 'A A A I S Q T Q R L S K K L G K V H G I R M S T K M V E D T P S I R G M I N K V K H L V M V E EM A A A K G V E Y L N D'},
 {'generated_text': 'A A A I S Q T Q R L S L L T E S F D R E Y L N R X A F A K X A V A T G KM A A A K G V E Y L N A F R I T F K P V A R K K'},
 {'generated_text': 'A A A I S L T G W L S V L I F T F S L A L V V W G R S G LM D A A N L S L T G W L T I M V L T F S L A V V W G R N G FM'},
 {'generated_text': 'A A A I S C M T G K L W N G E C V D H S C C E G L E V R K P P R I A Y G I H T C Y Y E CK F C P E G K C VK F G K V L K'},
 {'generated_text': 'A A A I S C M S D D S L V R L I SM L A A V V F L A L F L Y A I A T A D P G P G I S F L Y F R E E T K G K L Q L S K D P'},
 {'generated_text': 'A A A I S Q T Q R L S L L T E H K AM A A A K G A R E K I K L E S T A G T G H F Y T T T K N K K T M P E K M A I M K F D'},
 

In [52]:
tg("A A A I S C V G S K E C L P K C K A Q G C K S G K C", max_length=60, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'A A A I S C V G S K E C L P K C K A Q G C K S G K C M N G K C H C Y PM A A A K G V S C E E C K P K C R T Q G C K N D R'},
 {'generated_text': 'A A A I S C V G S K E C L P K C K A Q G C K S G K C M N G K C H C M K IG A A A N Q H L C G S H L V E A L Y L V C G E K'},
 {'generated_text': 'A A A I S C V G S K E C L P K C K A Q G C K S G K C M N K K C K C Y NG A A A N Q H L C G S H L V E A L Y L V C G E K G'},
 {'generated_text': 'A A A I S C V G S K E C L P K C K A Q G C K S G K C M N G K C M N G K C K C Y QM A A A K G V G K T R F K D K A R Q N T'},
 {'generated_text': 'A A A I S C V G S K E C L P K C K A Q G C K S G K C M N G K C H C Y YM A A A K L L E I L V C P L T K D R L P K H K A T'}]

In [53]:
tokenizer.encode('A A A')

[32, 317, 317]

In [54]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens([32, 317, 317]))

'A A A'

In [55]:
tokenizer.convert_ids_to_tokens([32, 317, 317])

['A', 'ĠA', 'ĠA']